In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
import pandas as pd
import numpy as np
from tqdm import tqdm
import wandb
import matplotlib.pyplot as plt
import torch

sys.path.append(os.path.join(os.getcwd(), '../symlie'))
sys.path.append(os.path.join(os.getcwd(), '..'))
from symlie.misc.utils import NumpyUtils, Results, tensor_operation, numpy_operation, Args
from symlie.misc.viz import plot2d
from symlie.model.setup import find_id_for_P, load_P_pred
from symlie.model.networks.linear import CalculatedP
from symlie.misc.viz import plot2d
from symlie.data.generate_2d import sine1d, sine2d, flower, mnist
from symlie.data.generate_data import datasets
from symlie.data.dataset import FlatDataset

from symlie.misc.utils_results import plot_data, assert_unique, pivot, plot_pivot, add_df_map_new, assert_columns_same, get_and_check_Ps, plot_seeds_and_Ps, rename_net, stringify_dict
from symlie.misc.wandb import update_results_df

In [2]:
# update_results_df()

df_map_columns = ['run_id', 'tags', 'data_kwargs', 'transform_kwargs', 'seed', 'data_dir']

df = pd.read_pickle('../logs/store/results_df.pkl')
print(df.shape)
print(df.columns)
df.head()

# df['tags'].astype(str).unique()

(543, 48)
Index(['lr', 'net', 'bias', 'name', 'seed', 'tags', 'test', 'A_low', 'n_val',
       'train', 'y_low', 'A_high', 'config', 'device', 'logger', 'n_test',
       'run_id', 'y_high', 'log_dir', 'n_train', 'predict', 'version',
       'y_multi', 'data_dir', 'eps_mult', 'n_splits', 'criterion', 'do_return',
       'earlystop', 'grid_size', 'noise_std', 'only_flip', 'batch_size',
       'max_epochs', 'data_kwargs', 'num_workers', 'out_features',
       'generate_data', 'model_summary', 'args_processed', 'do_return_model',
       'n_hidden_layers', 'transform_kwargs', 'use_P_from_noise',
       'persistent_workers', 'run_name', 'test_loss', 'n_classes'],
      dtype='object')


,lr,net,bias,name,seed,tags,test,A_low,n_val,train,...,model_summary,args_processed,do_return_model,n_hidden_layers,transform_kwargs,use_P_from_noise,persistent_workers,run_name,test_loss,n_classes
0,0.010,TrainP,False,None,3,"[new, noise]",True,NaN,1000,True,...,False,True,False,1.0,"{'eps_mult': [0, 1, 1, 1], 'only_flip': False}",False,True,dandy-donkey-1960,6.474450e-05,NaN
1,0.001,TrainP,False,None,3,"[new, noise]",True,NaN,1000,True,...,False,True,False,1.0,"{'eps_mult': [0, 1, 1, 1], 'only_flip': False}",False,True,swept-lion-1959,3.812419e-02,NaN
2,0.100,TrainP,False,None,3,"[new, noise]",True,NaN,1000,True,...,False,True,False,1.0,"{'eps_mult': [0, 1, 1, 1], 'only_flip': False}",False,True,atomic-planet-1958,1.422213e-07,NaN
3,0.001,TrainP,False,None,2,"[new, noise]",True,NaN,1000,True,...,False,True,False,1.0,"{'eps_mult': [0, 1, 1, 1], 'only_flip': False}",False,True,royal-mountain-1957,3.862118e-02,NaN
4,0.100,TrainP,False,None,2,"[new, noise]",True,NaN,1000,True,...,False,True,False,1.0,"{'eps_mult': [0, 1, 1, 1], 'only_flip': False}",False,True,glamorous-snowflake-1955,1.425501e-07,NaN


In [8]:
group_params = ['eps_mult', 'noise_std']
hyper_params  = ['lr']

datasets_predict = [
    'noise',
    # 'sine1d',
    # 'sine2d',
    # 'flower',
    # 'mnist',
]
    
default_log = [
    'noise',
]

d_pivots = {}
index_columns = ['dataset_name'] + group_params
dds = []
dds_mean = []
dds_std = []

mins = []

for dataset_name in datasets_predict:
    d = df[df['tags'].astype(str).str.contains(f"'{dataset_name}'")].reset_index(drop=True)
    d = d[~d['tags'].astype(str).str.contains(f'predict')].reset_index(drop=True)

    d = stringify_dict(d, group_params)
    ds = {group : d_group for group, d_group in d.groupby(group_params)} if group_params else {0 : d}


    for group, d in ds.items():

        d = d.reset_index(drop=True)

        for row in d['data_kwargs']:
            if 'grid_size' in row:
                row['grid_size'] = tuple(row['grid_size'])
        for row in d['transform_kwargs']:
            if 'eps_mult' in row:
                row['eps_mult'] = [float(x) for x in row['eps_mult']]

        d = d.drop_duplicates(subset=hyper_params + ['seed', 'test_loss']) #TODO: why is this necessary?
        map_kwargs = assert_columns_same(d, ['data_kwargs', 'transform_kwargs', 'data_dir'])

        d_pivot = pivot(d, columns = hyper_params)
        # d_pivot = rename_net(d_pivot)
        d_pivots[(dataset_name, group)] = dict(d_pivot=d_pivot, map_kwargs=map_kwargs)

        dd = pd.DataFrame(data = [f'{mean:.2e} ± {std:.2e}' for mean, std in zip(d_pivot.mean().values, d_pivot.std().values)], index = d_pivot.columns).T
        dd_mean = pd.DataFrame(data = [mean for mean in d_pivot.mean().values], index = d_pivot.columns).T
        dd_std = pd.DataFrame(data = [std for std in d_pivot.std().values], index = d_pivot.columns).T

        for dd_i, dds_i in zip([dd, dd_mean, dd_std], [dds, dds_mean, dds_std]):

            dd_i[index_columns] = [dataset_name] + list(group)
            dd_i = dd_i.set_index(index_columns)
            dds_i.append(dd_i)

        # continue


        hparams_min = d_pivot.mean().idxmin()
        assert len(hparams_min) == 2
        df_map_new = d[d[hyper_params[0]] == hparams_min[1]][df_map_columns]
        add_df_map_new(df_map_new)
        # Ps = get_and_check_Ps(df_map_new['seed'].unique(), map_kwargs)
        # plot2d(Ps, l = 2, max_grid = 40)

        print(f'{hparams_min=}')
        mins.append((dataset_name, group, hparams_min))
        logx = True if dataset_name in default_log else False
        suptitle = dataset_name + '\n' + ', '.join([f'{group_param} = {group_el}' for group_param, group_el in zip(group_params, group)]) if group != 0 else dataset_name
#         plot_pivot(d_pivot=d_pivot, legend_loc = 'lower left', logx=logx, suptitle = suptitle, figsize = (4, 2))

        break

# ddd = pd.concat(dds)
# ddd_mean = pd.concat(dds_mean)
# ddd_std = pd.concat(dds_std)
# ddd

hparams_min=('test_loss', 0.1)


In [9]:
d_pivot

test_loss                    
lr       0.001     0.010     0.100
seed                              
1     0.000347  0.000062  0.000009
2     0.000391  0.000061  0.000008
3     0.000312  0.000061  0.000008

In [ ]:
for logy in [False, True]:
    ddd_mean.plot(kind='bar', yerr = ddd_std, figsize = (10, 5), logy = logy, rot = 45)
    plt.show()

In [ ]:
d = df.copy()

d = d[~d['tags'].astype(str).str.contains('predict')].reset_index(drop=True)
d = stringify_dict(d, group_params)

dd = d.groupby(['data_dir'] + group_params + hyper_params + ['seed']).count()
print(dd[dd['test_loss'] > 1].index.values)

# d = d.pivot(index = ['data_dir'] + group_params , columns = hyper_params +  ['seed'], values = 'test_loss')

d = pivot(d, index = ['data_dir'] + group_params, columns = hyper_params + ['seed'], values = 'test_loss')

# d[d.isna().any(axis=0)]

d

In [ ]:
df_map = pd.read_pickle('../logs/store/map_df.pkl')
df_map

## Map

In [ ]:
def filter_on_tags(df, tags):
    return df[df['tags'].astype(str).str.contains('|'.join(tags))]

In [ ]:
df_map = pd.read_pickle('../logs/store/map_df.pkl')

df_map_sync = filter_on_tags(df_map, tags = ['sine1d', 'noise'])
df_map_sync = df_map_sync.reset_index(drop=True)
df_map_sync

In [ ]:
sync_run_ids = list(df_map_sync['run_id'].values)

store_dir = '../logs/store'
# write the run_ids to a txt file
with open(os.path.join(store_dir, 'sync_run_ids.txt'), 'w') as f:
    f.write(f"{{{','.join(sync_run_ids)}}}")

## Let's check Ps

In [ ]:
# d = df_map_sync
d = df_map

d = filter_on_tags(d, tags = ['noise'])

transform_kwargs = {'eps_mult': [0.0, 1.0, 1.0, 1.0], 'only_flip': False}
d = d[d['transform_kwargs'] == transform_kwargs]

map_kwargs = assert_columns_same(d, ['data_kwargs', 'transform_kwargs', 'data_dir'])
map_kwargs

In [ ]:
Ps = get_and_check_Ps(d['seed'].unique(), map_kwargs, use_P_from_noise = True)
# Ps = get_and_check_Ps([1], map_kwargs)
plot2d(Ps, l = 2, max_grid = 40)